# Generate log file

In [4]:
%matplotlib inline
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import re

import pandas as pd

## List all logs files

In [5]:
import glob
#files = glob.glob('../results/logs/**/events.out.tfevents.*', recursive=True)
files = glob.glob('../results/logs_2019_03_09 _RBM_RBMCF_100_250_500_1000/**/events.out.tfevents.*', recursive=True)

len(files), files[0:3]

(136,
 ['../results/logs_2019_03_09 _RBM_RBMCF_100_250_500_1000/kfold=0-Adam/kfold-intern=0/batch_size=10/class=RBMCF/visible_size=702/hidden_size=250/regularization=NoRegularization-0.0/learning_rate=Adam-0.001-0.9-0.999-1e-08/sampling_method=CD-1/momentum=0.5/1552239105.4836464/events.out.tfevents.1552239109.paulo-notebook',
  '../results/logs_2019_03_09 _RBM_RBMCF_100_250_500_1000/kfold=0-Adam/kfold-intern=0/batch_size=10/class=RBMCF/visible_size=702/hidden_size=250/regularization=NoRegularization-0.0/learning_rate=Adam-0.001-0.9-0.999-1e-08/sampling_method=CD-1/momentum=0/1552237344.970886/events.out.tfevents.1552237347.paulo-notebook',
  '../results/logs_2019_03_09 _RBM_RBMCF_100_250_500_1000/kfold=0-Adam/kfold-intern=0/batch_size=10/class=RBMCF/visible_size=702/hidden_size=500/regularization=NoRegularization-0.0/learning_rate=Adam-0.001-0.9-0.999-1e-08/sampling_method=CD-1/momentum=0.5/1552244451.3214648/events.out.tfevents.1552244454.paulo-notebook'])

## Read a file

In [6]:
file = files[0]

def extract_information(file):
    ea = EventAccumulator(file)
    ea.Reload()
    
    scalars = ea.Tags()['scalars']
    first_scalar = ea.Scalars(scalars[0])
    
    dataframes = pd.DataFrame(first_scalar)[['wall_time', 'step']]
    
    for df_name in ea.Tags()['scalars']:
        df = pd.DataFrame(ea.Scalars(df_name))
        series = df['value'].rename(df_name)
        dataframes = pd.concat([dataframes, series], axis=1)
    
    del ea

    return dataframes

dfs = extract_information(file)
print(dfs.shape[0], 'evaluates')
print(dfs.shape[1], 'data columns')

dfs.head()

151 evaluates
44 data columns


,wall_time,step,measure/reconstruction/error,measure/activation/mean,measure/activation/std,measure/parameters/W/mean,measure/parameters/b_h/mean,measure/parameters/b_v/mean,measure/reconstruction_1/hamming,details/measure/top-1/evaluate-0/train,...,details/measure/top-5/evaluate-5/validation,measure/evaluate/top-5/train,measure/evaluate/top-5/validation,measure/evaluate/Free/mean_free_energy_train,measure/evaluate/Free/mean_free_energy_validation,measure/evaluate/Free/ratio_mean_free_energy,measure/evaluate/Free/mean_free_energy_gap,measure/evaluate/Free/mean_free_energy_noisy,measure/evaluate/reconstruction/RMSE_train,measure/evaluate/reconstruction/RMSE_validation
0,1.552239e+09,1,0.123864,5.864385,5.772261,-0.004844,-0.141609,0.005498,2.862410,0.330481,...,0.663158,0.429449,0.432456,-168.187424,-167.987900,0.998814,15.292770,-152.695129,0.088351,0.088510
1,1.552239e+09,11,0.122772,6.040158,5.605364,-0.027167,-1.314725,-0.012838,2.935484,0.330481,...,0.684211,0.446785,0.448246,-67.161118,-66.767548,0.994140,15.590923,-51.176624,0.086256,0.086481
2,1.552239e+09,21,0.117910,5.865043,4.822471,-0.045995,-1.730996,-0.029660,2.983542,0.331139,...,0.707895,0.482335,0.478509,-48.165077,-47.587521,0.988009,15.519356,-32.068165,0.082112,0.082509
3,1.552239e+09,31,0.114512,6.048058,4.759377,-0.061905,-1.952907,-0.040953,2.959184,0.437788,...,0.668421,0.511960,0.485965,-41.222622,-40.310238,0.977867,15.953270,-24.356968,0.078782,0.079676
4,1.552239e+09,41,0.111987,6.054641,4.422952,-0.076068,-2.098953,-0.048771,2.951284,0.441738,...,0.671053,0.537964,0.478947,-38.018906,-36.618214,0.963158,16.586512,-20.031702,0.076307,0.077797


In [7]:
def extract_columns(file):
    regex = r'(.+?)=(.+?)\/'
    test_str = file.replace('../results/logs/', '')
    matches = re.finditer(regex, test_str, re.MULTILINE)

    columns = {match.groups()[0]: match.groups()[1] for match in matches}
    columns['path'] = test_str
    return columns

def populate_columns(file, dataframe):
    columns = extract_columns(file)
    
    for k, v in columns.items():
        dataframe[k] = v

    return dataframe

In [8]:
def generate_logs_files(files):
    data = []

    for index, file in enumerate(files):
        print(f'{index+1} of {len(files)}')
        information = extract_information(file)
        information = populate_columns(file, information)
        
        data.append(information)
        
    return pd.concat(data)

data = generate_logs_files(files)

1 of 136
2 of 136
3 of 136
4 of 136
5 of 136
6 of 136
7 of 136
8 of 136
9 of 136
10 of 136
11 of 136
12 of 136
13 of 136
14 of 136
15 of 136
16 of 136
17 of 136
18 of 136
19 of 136
20 of 136
21 of 136
22 of 136
23 of 136
24 of 136
25 of 136
26 of 136
27 of 136
28 of 136
29 of 136
30 of 136
31 of 136
32 of 136
33 of 136
34 of 136
35 of 136
36 of 136
37 of 136
38 of 136
39 of 136
40 of 136
41 of 136
42 of 136
43 of 136
44 of 136
45 of 136
46 of 136
47 of 136
48 of 136
49 of 136
50 of 136
51 of 136
52 of 136
53 of 136
54 of 136
55 of 136
56 of 136
57 of 136
58 of 136
59 of 136
60 of 136
61 of 136
62 of 136
63 of 136
64 of 136
65 of 136
66 of 136
67 of 136
68 of 136
69 of 136
70 of 136
71 of 136
72 of 136
73 of 136
74 of 136
75 of 136
76 of 136
77 of 136
78 of 136
79 of 136
80 of 136
81 of 136
82 of 136
83 of 136
84 of 136
85 of 136
86 of 136
87 of 136
88 of 136
89 of 136
90 of 136
91 of 136
92 of 136
93 of 136
94 of 136
95 of 136
96 of 136
97 of 136
98 of 136
99 of 136
100 of 136
101 of 1

In [9]:
data.index.name = 'evaluate'
data.head(5)

,wall_time,step,measure/reconstruction/error,measure/activation/mean,measure/activation/std,measure/parameters/W/mean,measure/parameters/b_h/mean,measure/parameters/b_v/mean,measure/reconstruction_1/hamming,details/measure/top-1/evaluate-0/train,...,kfold-intern,batch_size,class,visible_size,hidden_size,regularization,learning_rate,sampling_method,momentum,path
evaluate,,,,,,,,,,,,,,,,,,,,,
0,1.552239e+09,1,0.123864,5.864385,5.772261,-0.004844,-0.141609,0.005498,2.862410,0.330481,...,0,10,RBMCF,702,250,NoRegularization-0.0,Adam-0.001-0.9-0.999-1e-08,CD-1,0.5,../results/logs_2019_03_09 _RBM_RBMCF_100_250_...
1,1.552239e+09,11,0.122772,6.040158,5.605364,-0.027167,-1.314725,-0.012838,2.935484,0.330481,...,0,10,RBMCF,702,250,NoRegularization-0.0,Adam-0.001-0.9-0.999-1e-08,CD-1,0.5,../results/logs_2019_03_09 _RBM_RBMCF_100_250_...
2,1.552239e+09,21,0.117910,5.865043,4.822471,-0.045995,-1.730996,-0.029660,2.983542,0.331139,...,0,10,RBMCF,702,250,NoRegularization-0.0,Adam-0.001-0.9-0.999-1e-08,CD-1,0.5,../results/logs_2019_03_09 _RBM_RBMCF_100_250_...
3,1.552239e+09,31,0.114512,6.048058,4.759377,-0.061905,-1.952907,-0.040953,2.959184,0.437788,...,0,10,RBMCF,702,250,NoRegularization-0.0,Adam-0.001-0.9-0.999-1e-08,CD-1,0.5,../results/logs_2019_03_09 _RBM_RBMCF_100_250_...
4,1.552239e+09,41,0.111987,6.054641,4.422952,-0.076068,-2.098953,-0.048771,2.951284,0.441738,...,0,10,RBMCF,702,250,NoRegularization-0.0,Adam-0.001-0.9-0.999-1e-08,CD-1,0.5,../results/logs_2019_03_09 _RBM_RBMCF_100_250_...


In [10]:
data.to_csv('extracted_log-rbm-part-one.csv')